### 导入自定义策略，开始回测

In [1]:
from tradition.base import TwoSMA_Official

DATASET = r"..\data\000001_daily_qfq.csv"
START_DATE = "2025-01-01"
END_DATE = "2025-06-30"

STRATAGE = TwoSMA_Official

e:\miniconda\envs\train\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\miniconda\envs\train\lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
import pandas as pd
from backtesting import set_bokeh_output

### 设置浏览器加载图像
set_bokeh_output(notebook=False)

# 1. 读取CSV文件
df = pd.read_csv(DATASET)

# 2. 重命名列为英文
df.columns = [
    "Date", "StockCode", "Open", "Close", "High", "Low",
    "Volume", "Turnover", "Amplitude", "ChangePct", "ChangeAmt", "TurnoverRate"
]

# 3. 将日期设置为索引，并转换为 datetime 类型
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", inplace=True)

# 4. 筛选日期范围内的数据
filtered_df = df.loc[START_DATE:END_DATE]
filtered_df = filtered_df[["Open", "Close", "High", "Low", "Volume"]]

# 6. 打印结果
filtered_df

,Open,Close,High,Low,Volume
Date,,,,,
2025-01-02,11.37,11.07,11.41,11.03,1819597
2025-01-03,11.08,11.02,11.18,11.00,1154680
2025-01-06,11.02,11.08,11.12,10.86,1085536
2025-01-07,11.06,11.15,11.17,11.01,747863
2025-01-08,11.14,11.14,11.27,11.04,1062386
...,...,...,...,...,...
2025-06-24,11.90,11.93,12.01,11.84,1308525
2025-06-25,11.93,12.06,12.08,11.83,1675116
2025-06-26,12.05,12.41,12.48,12.01,2174578


In [11]:
from backtesting import Backtest
from datetime import datetime

bt = Backtest(filtered_df,
              STRATAGE,
              cash=100000, 
              spread=0,                 # 默认值，卖出价就是买入价，没有价差
              commission=.002,          # 0.15%-0.20%适合A股
              margin=1,                 # 默认值，似乎只有加杠杆会用到
              trade_on_close=False,     # 用当日收盘价或次日开盘价交易，当日依据收盘价生成信号，次日挂单
              hedging=False,            # A股不允许同时持有多空仓，不能做空，设为 False
              exclusive_orders=True,    # 通常一个策略只操作一只股票，或者一次只持有一只
              finalize_trades=True)    # 回测结束时强制平仓，统计完整收益，Optional
stats = bt.run()
print(stats)
bt.plot(filename=f"../result/{STRATAGE.__name__}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.html")

Start                     2025-01-02 00:00:00
End                       2025-06-30 00:00:00
Duration                    179 days 00:00:00
Exposure Time [%]                    58.11966
Equity Final [$]                 109964.06104
Equity Peak [$]                  112634.94632
Commissions [$]                    1211.45896
Return [%]                            9.96406
Buy & Hold Return [%]                 9.72727
Return (Ann.) [%]                    22.70092
Volatility (Ann.) [%]                14.97156
CAGR [%]                             14.30723
Sharpe Ratio                          1.51627
Sortino Ratio                         2.85915
Calmar Ratio                          3.37003
Alpha [%]                             5.22498
Beta                                  0.48719
Max. Drawdown [%]                    -6.73613
Avg. Drawdown [%]                    -2.44154
Max. Drawdown Duration      105 days 00:00:00
Avg. Drawdown Duration       25 days 00:00:00
# Trades                          

GridPlot(id='p2475', ...)

In [12]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,Commission,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_SMA(C,10)","Exit_SMA(C,10)","Entry_SMA(C,20)","Exit_SMA(C,20)"
0,8966,25,41,11.13,11.30,None,None,1122.00524,402.21476,0.011243,2025-02-14,2025-03-10,24 days,None,11.064,11.211,11.0290,11.2295
1,8806,45,54,11.46,11.01,None,None,-4358.44164,395.74164,-0.043188,2025-03-14,2025-03-27,13 days,None,11.324,11.137,11.2985,11.2085
2,9076,76,116,10.64,12.14,None,None,13200.49744,413.50256,0.136695,2025-04-29,2025-06-30,62 days,None,10.673,11.967,10.6550,11.7295
